In [2]:
import numpy as np
import pandas as pd
from bs4  import BeautifulSoup
import requests
import time
from datetime import datetime 
import warnings
warnings.filterwarnings("ignore")
import sys
import json
from json import loads, dumps
import lxml
from requests import ConnectionError, ReadTimeout, ConnectTimeout, HTTPError, Timeout
import xml
import re
from natsort import natsorted
import xml.etree.ElementTree as ET
import xmltodict
from xml.parsers.expat import ExpatError
from requests.exceptions import ChunkedEncodingError

In [7]:
import hockey_scraper

In [14]:
# USER_PATH = '/Users/jamesbenasuli/Desktop/nhl-predictons/hockey_scraper_data/'

# hockey_scraper.nhl.live_scrape.set_docs_dir(USER_PATH)
# hockey_scraper.nhl.scrape_functions.print_errors(detailed=True)

In [8]:
# use schedule function to get 4/4/23 game ids
sched_df = hockey_scraper.scrape_schedule("2023-04-04", "2023-04-05")
sched_df

Scraping the schedule between 2023-04-04 and 2023-04-05


,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status
0,2022021229,2023-04-04,FLA Live Arena,FLA,BUF,2023-04-04 23:00:00,2,1,Final
1,2022021230,2023-04-04,Scotiabank Arena,TOR,CBJ,2023-04-04 23:00:00,4,2,Final
2,2022021231,2023-04-04,Centre Bell,MTL,DET,2023-04-04 23:00:00,0,5,Final
3,2022021232,2023-04-04,PNC Arena,CAR,OTT,2023-04-04 23:00:00,3,2,Final
4,2022021233,2023-04-04,Prudential Center,N.J,PIT,2023-04-04 23:00:00,5,1,Final
5,2022021234,2023-04-04,Enterprise Center,STL,PHI,2023-04-05 00:00:00,4,2,Final
6,2022021235,2023-04-04,Bridgestone Arena,NSH,VGK,2023-04-05 00:00:00,3,2,Final
7,2022021236,2023-04-04,Scotiabank Saddledome,CGY,CHI,2023-04-05 01:00:00,3,4,Final
8,2022021238,2023-04-04,Rogers Arena,VAN,SEA,2023-04-05 02:00:00,2,5,Final
9,2022021237,2023-04-04,Crypto.com Arena,L.A,EDM,2023-04-05 02:30:00,1,3,Final


In [18]:
# explicitly single game scraper
# test with 2022021230
test_game_id = '2022021230'
date = "2023-04-04"
test_game = hockey_scraper.nhl.game_scraper.scrape_game(test_game_id, date, if_scrape_shifts=True)


Scraping Game  2022021230 2023-04-04


In [26]:
pbp_test, shifts_test = test_game


In [27]:
pbp_test

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
0,21230,2023-04-04,1,PSTR,Period Start- Local time: 7:08 EDT,0:00,0.0,5x5,None,,...,0,0,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,NaN,NaN,Sheldon Keefe,Brad Larsen
1,21230,2023-04-04,1,FAC,TOR won Neu. Zone - CBJ #38 JENNER vs TOR #64 ...,0:00,0.0,5x5,Neu,,...,0,0,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,0.0,0.0,Sheldon Keefe,Brad Larsen
2,21230,2023-04-04,1,HIT,"TOR #12 ASTON-REESE HIT CBJ #86 MARCHENKO, Neu...",0:13,13.0,5x5,Neu,,...,0,0,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,3.0,-39.0,Sheldon Keefe,Brad Larsen
3,21230,2023-04-04,1,BLOCK,"CBJ #13 GAUDREAU BLOCKED BY TOR #22 MCCABE, B...",0:26,26.0,5x5,Def,BACKHAND,...,0,0,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,83.0,-11.0,Sheldon Keefe,Brad Larsen
4,21230,2023-04-04,1,STOP,ICING,0:43,43.0,5x5,None,,...,0,0,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,NaN,NaN,Sheldon Keefe,Brad Larsen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,21230,2023-04-04,3,STOP,OFFSIDE,19:02,1142.0,5x5,None,,...,2,4,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,NaN,NaN,Sheldon Keefe,Brad Larsen
357,21230,2023-04-04,3,FAC,TOR won Neu. Zone - CBJ #21 DUNNE vs TOR #28 L...,19:02,1142.0,5x5,Neu,,...,2,4,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,20.0,22.0,Sheldon Keefe,Brad Larsen
358,21230,2023-04-04,3,MISS,"TOR #28 LAFFERTY, Wrist, Wide of Net, Off. Zon...",19:16,1156.0,5x5,Off,WRIST SHOT,...,2,4,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,-67.0,19.0,Sheldon Keefe,Brad Larsen
359,21230,2023-04-04,3,PEND,Period End- Local time: 9:31 EDT,20:00,1200.0,5x5,None,,...,2,4,JET GREAVES,8482982.0,JOSEPH WOLL,8479361.0,NaN,NaN,Sheldon Keefe,Brad Larsen


In [28]:
shifts_test

,Game_Id,Period,Team,Player,Player_Id,Start,End,Duration,Date
0,21230,1,TOR,JUSTIN HOLL,8475718,0.0,52.0,52.0,2023-04-04
1,21230,1,CBJ,JOHNNY GAUDREAU,8476346,0.0,36.0,36.0,2023-04-04
2,21230,1,CBJ,BOONE JENNER,8476432,0.0,34.0,34.0,2023-04-04
3,21230,1,TOR,JAKE MCCABE,8476931,0.0,53.0,53.0,2023-04-04
4,21230,1,TOR,SAM LAFFERTY,8478043,0.0,37.0,37.0,2023-04-04
...,...,...,...,...,...,...,...,...,...
741,21230,3,CBJ,JET GREAVES,8482982,1137.0,1200.0,63.0,2023-04-04
742,21230,3,CBJ,HUNTER MCKOWN,8484125,1137.0,1200.0,63.0,2023-04-04
743,21230,3,CBJ,MARCUS BJORK,8483620,1170.0,1200.0,30.0,2023-04-04
744,21230,3,CBJ,BILLY SWEEZEY,8482399,1171.0,1200.0,29.0,2023-04-04


In [ ]:
# def to_csv(game):
#     """
#     Store each game DataFrame in a file

#     :param game: LiveGame object

#     :return: None
#     """

#     # If the game:
#     # 1. Started - We recorded at least one event
#     # 2. Not in Intermission
#     # 3. Not Over
#     if game.is_ongoing():
#         # Print the description of the last event
#         print(game.game_id, "->", game.pbp_df.iloc[-1]['Description'])

#         # Store in CSV files
#         game.pbp_df.to_csv(f"../hockey_scraper_data/{game.game_id}_pbp.csv", sep=',')
#         game.shifts_df.to_csv(f"../hockey_scraper_data/{game.game_id}_shifts.csv", sep=',')

# if __name__ == "__main__":
#     # B4 we start set the directory to store the files
#     hs.live_scrape.set_docs_dir("../hockey_scraper_data")

#     # Scrape the info for all the games on 2018-11-15
#     games = hs.ScrapeLiveGames("2018-11-15", if_scrape_shifts=True, pause=20)

#     # While all the games aren't finished
#     while not games.finished():
#         # Update for all the games currently being played
#         games.update_live_games(sleep_next=True)

#         # Go through every LiveGame object and apply some function
#         # You can of course do whatever you want here.
#         for game in games.live_games:
#             to_csv(game)

In [ ]:
# Scrape the info for all the games on 2018-11-15 games = 
# hs.ScrapeLiveGames("2018-11-15", if_scrape_shifts=True, pause=15, game_ids=[2018020280, 2018020281])
games = hs.ScrapeLiveGames("2018-11-15", if_scrape_shifts=True, pause=15)

while not games.finished():
    # Go through every LiveGame object
    for game in games.live_games:
        # Scrape each game individually
        game.scrape()

        # Apply some function to every game
        to_csv(game)

    # Pause after each scraping chunk
    time.sleep(15)

In [37]:
# Scrapes the 2023 season without shifts and stores the data in a Csv file (both are equivalent!!!)
# scraped_data = hockey_scrsaper.scrape_seasons([2023], False, docs_dir='/Users/jamesbenasuli/Desktop/nhl-predictons/hockey_scraper_data')

hockey_scraper.nhl.scrape_functions.scrape_seasons([2023], if_scrape_shifts=False, data_format='csv', preseason=False, rescrape=False, 
                                                #docs_dir='/Users/jamesbenasuli/Desktop/nhl-predictons/hockey_scraper_data', verbose=False)

In [38]:
# allow to use its own directory and set rescrape to True
#  no shifts
hockey_scraper.nhl.scrape_functions.scrape_seasons([2020, 2021, 2022], if_scrape_shifts=False, data_format='csv', preseason=False, rescrape=False, 
                                                docs_dir=True, verbose=False)

Scraping Game  2020020001 2021-01-13
Scraping Game  2020020003 2021-01-13
Scraping Game  2020020002 2021-01-13
Scraping Game  2020020004 2021-01-13
Scraping Game  2020020005 2021-01-13
Scraping Game  2020020006 2021-01-14
Scraping Game  2020020007 2021-01-14
Scraping Game  2020020008 2021-01-14
Scraping Game  2020020010 2021-01-14
Scraping Game  2020020011 2021-01-14
Scraping Game  2020020012 2021-01-14
Scraping Game  2020020013 2021-01-14
Scraping Game  2020020014 2021-01-14
Scraping Game  2020020015 2021-01-14
Scraping Game  2020020016 2021-01-14
Scraping Game  2020020017 2021-01-15
Scraping Game  2020020018 2021-01-15
Scraping Game  2020020019 2021-01-15
Scraping Game  2020020021 2021-01-15
Scraping Game  2020020022 2021-01-15
Scraping Game  2020020023 2021-01-16
Scraping Game  2020020030 2021-01-16
Scraping Game  2020020029 2021-01-16
Scraping Game  2020020024 2021-01-16
Scraping Game  2020020025 2021-01-16
Scraping Game  2020020027 2021-01-16
Scraping Game  2020020026 2021-01-16
S

KeyboardInterrupt: 

In [31]:
# scrape function tests
# hockey_scraper.nhl.scrape_functions.scrape_games(games, if_scrape_shifts, docs_dir=False, verbose=False)
games_list = ['2022021231', '2022021232']
# datae = "2023-04-04"
hockey_scraper.nhl.scrape_functions.scrape_games(games=games_list, if_scrape_shifts=False, data_format='csv', rescrape=False, 
                                                docs_dir='/Users/jamesbenasuli/Desktop/nhl-predictons/hockey_scraper_data',
                                                verbose=False)

Scraping Game  2022021231 2023-04-04
Scraping Game  2022021232 2023-04-04

---> nhl pbp data deposited in file - /Users/jamesbenasuli/Desktop/nhl-predictons/hockey_scraper_data/csvs/nhl_pbp_1680748174.csv


In [10]:
import datetime
import os
import requests
import pandas as pd
import time

def get_today_sched(date):
    '''
    this function gets the schedule of games that are set to be played on the
    date pased to it in the NHL
    Input:
    date - date to get schedule
    Ouput:
    games - dictionary of games to be played
    '''

    url = f'https://statsapi.web.nhl.com/api/v1/schedule?date={date}'

    req = requests.get(url)
    schedule_dict = req.json()

    today_games = {}

#if there are no games exit the function with an empty dataframe
    if not schedule_dict['dates']:
        return pd.DataFrame()

#parse the schedule to create a dataframe to feed to the prediction model
    for x in schedule_dict['dates']:
        for game in x['games']:
            
            #package function
            today_games[game['gamePk']] = {}
            # add season id
            today_games[game['gamePk']]['season_id'] = game['season']
            today_games[game['gamePk']]['date'] = date
            today_games[game['gamePk']]['home_team'] = game['teams']['home']['team']['name']
            today_games[game['gamePk']]['home_team_id'] = game['teams']['home']['team']['id']
            today_games[game['gamePk']]['away_team'] = game['teams']['away']['team']['name']
            today_games[game['gamePk']]['away_team_id'] = game['teams']['away']['team']['id']
# add score info
            today_games[game['gamePk']]['home_team_score'] = game['teams']['home']['score']
            today_games[game['gamePk']]['away_team_score'] = game['teams']['away']['score']
# current record of both teams
            today_games[game['gamePk']]['home_team_wins'] = game['teams']['home']['leagueRecord']['wins']
            today_games[game['gamePk']]['home_team_losses'] = game['teams']['home']['leagueRecord']['losses']
            today_games[game['gamePk']]['home_team_otl'] = game['teams']['home']['leagueRecord']['ot']
            today_games[game['gamePk']]['away_team_wins'] = game['teams']['away']['leagueRecord']['wins']
            today_games[game['gamePk']]['away_team_losses'] = game['teams']['away']['leagueRecord']['losses']
            today_games[game['gamePk']]['away_team_otl'] = game['teams']['away']['leagueRecord']['ot']

#turn dictionary of daily games to a dataframe:
    daily_games_df = pd.DataFrame.from_dict(today_games, orient='index')
    daily_games_df = daily_games_df.reset_index()
    daily_games_df.columns = ['game_id', 'season_id', 'game_date', 'home_team', 'home_team_id', 'away_team', 'away_team_id', 'home_team_score', 'away_team_score',
                              'home_team_wins', 'home_team_losses', 'home_team_otl', 'away_team_wins', 'away_team_losses', 'away_team_otl']

    return daily_games_df

# def main():
#     get_today_sched('2018-10-04')

# if __name__ == '__main__':
#     main()

In [11]:
get_today_sched('2023-04-11')

,game_id,season_id,game_date,home_team,home_team_id,away_team,away_team_id,home_team_score,away_team_score,home_team_wins,home_team_losses,home_team_otl,away_team_wins,away_team_losses,away_team_otl
0,2022021285,20222023,2023-04-11,New Jersey Devils,1,Buffalo Sabres,7,6,2,51,22,8,40,33,7
1,2022021287,20222023,2023-04-11,Philadelphia Flyers,4,Columbus Blue Jackets,29,4,3,30,38,13,24,47,9
2,2022021288,20222023,2023-04-11,Carolina Hurricanes,12,Detroit Red Wings,17,4,1,51,21,9,35,36,10
3,2022021289,20222023,2023-04-11,Tampa Bay Lightning,14,Toronto Maple Leafs,10,3,4,45,30,6,49,21,11
4,2022021290,20222023,2023-04-11,Boston Bruins,6,Washington Capitals,15,5,2,64,12,5,35,37,9
5,2022021286,20222023,2023-04-11,Pittsburgh Penguins,5,Chicago Blackhawks,16,2,5,40,31,10,26,49,6
6,2022021291,20222023,2023-04-11,Minnesota Wild,30,Winnipeg Jets,52,1,3,46,25,10,46,32,3
7,2022021292,20222023,2023-04-11,Colorado Avalanche,21,Edmonton Oilers,22,1,2,49,24,7,49,23,9
8,2022021293,20222023,2023-04-11,Vegas Golden Knights,54,Seattle Kraken,55,4,1,50,22,9,46,27,8
9,2022021294,20222023,2023-04-11,Anaheim Ducks,24,Vancouver Canucks,23,2,3,23,46,12,37,37,7


NameError: name 'daily_games_df' is not defined